In [15]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import mosek
from scipy.stats import lognorm

In [7]:
def profit_vec(y, D, p, c, s, l):
    return p * np.minimum(y, D) - c * y + s * np.maximum(y - D, 0) - l * np.maximum(D - y, 0)

def dprofit_vec(y, D, p, c, s, l):
    return np.where(D >= y, p + l - c, s - c)

def y_opt(alpha,p,c,s,l, mu,sigma):
    E = c-s
    U = p+l-c
    V = p-c
    F1 = lognorm.ppf(U*alpha/(E+U), sigma, scale=np.exp(mu))
    F2 = lognorm.ppf((E*(1-alpha)+U)/(E+U), sigma, scale=np.exp(mu))
    return((E+V)/(E+U)*F1+(U-V)/(E+U)*F2)

In [9]:
def f_explog_vec(x, a, b):
    e = np.exp(1)
    c1 = 1 / (b**2 * (a**2 + a) * np.exp(a - 1))
    c2 = 1 - np.exp(a) * (a * b + 1) * c1
    c3 = -np.exp(a + 1) * c1

    output = np.empty_like(x, dtype=float)
    pos = x > 0
    term = (x[pos] + e) * np.exp(a * np.log(x[pos] + e)**b)
    output[pos] = c1 * term + c2 * x[pos] + c3
    output[~pos] = np.exp(x[~pos]) - 1

    return output

def df_explog_vec(x, a, b):
    e = np.exp(1)
    
    c1 = 1 / (b**2 * (a**2 + a) * np.exp(a - 1))
    c2 = 1 - np.exp(a) * (a * b + 1) * c1

    output = np.empty_like(x, dtype=float)
    pos = x > 0

    term = np.exp(a * (np.log(x[pos] + e))**b) * (a * b * (np.log(x[pos] + e))**(b - 1) + 1)
    output[pos] = c1 * term + c2
    output[~pos] = np.exp(x[~pos])

    return output

def ddf_explog_vec(x, a, b):
    e = np.exp(1)
    
    c1 = 1 / (b**2 * (a**2 + a) * np.exp(a - 1))

    output = np.empty_like(x, dtype=float)
    pos = x > 0

    term1 = a * b / (x[pos] + e) * np.exp(a * np.log(x[pos] + e)**b)
    term2 = (b - 1) * np.log(x[pos] + e)**(b - 2) + np.log(x[pos] + e)**(b - 1) + a * b * np.log(x[pos] + e)**(2 * b - 2)
    output[pos] = c1 * term1 * term2
    output[~pos] = np.exp(x[~pos])

    return output

def CVaR_vec(x,alpha):
    return np.maximum(1/alpha*x,0)

def dCVaR_vec(x, alpha):
    return np.where(x > 0, 1 / alpha, 0)

def cvar_fast(alpha,x,f):
    n = len(f)
    order = np.argsort(x)
    x = np.sort(x)
    f = f[order]
    csum = np.cumsum(f)
    if csum[0] >= alpha:
        return(-x[0])
    else:
        k_max = np.max(np.where((csum < alpha)==True))
    Ex = np.sum(np.multiply(x[0:k_max+1],f[0:k_max+1]))
    last = (alpha - csum[k_max])*x[k_max+1]
    return(-1/alpha*(Ex+last))

In [11]:
def ellipsoid_explog_vec(x0, max_iter,r_e,para):
    n = len(x0)
    x = x0
    P = np.identity(n)*r_e
    a,b,alpha,r,c,p,s,l,D = para
    f_best = np.inf
    x_opt = x0
    N = len(D)
    for steps in range(max_iter):
        if x[2] >= 0:
            the1 = x[0]
            the2 = x[1]
            lbda = x[2]
            y = x[3]
            g = np.zeros(4)
            arg = CVaR_vec(the2-profit_vec(y,D,p,c,s,l),alpha)
            f_obj = np.sum(lbda * f_explog_vec((arg+the1)/lbda,a,b))/N -the1-the2+lbda*r
            s1 = np.sum(df_explog_vec((arg+the1)/lbda,a,b))
            s2 = np.sum(df_explog_vec((arg+the1)/lbda,a,b)* dCVaR_vec(the2-profit_vec(y,D,p,c,s,l),alpha))
            s3 = np.sum(f_explog_vec((arg+the1)/lbda,a,b) - df_explog_vec((arg+the1)/lbda,a,b) * ((arg+the1)/lbda))
            s4 = np.sum(df_explog_vec((arg+the1)/lbda,a,b) * dCVaR_vec(the2-profit_vec(y,D,p,c,s,l),alpha)* -dprofit_vec(y,D,p,c,s,l))
            
            g[0] = -1 + s1/N
            g[1] = -1 + s2/N
            g[2] = r + s3/N
            g[3] = s4/N
            n_g =np.sqrt((g.dot(P)).dot(g)) 
            g_n = g/n_g
            if f_obj < f_best and x[2] >= 0:
                f_best = f_obj
                x_opt = x
                x_grad = g
            #print('obj', f_best, 'sol', x_opt, np.max(P))
        else:
            g = np.array([0,0,-1,0])
            #print('constr_g', (g.dot(P)).dot(g))
            n_g =np.sqrt((g.dot(P)).dot(g)) 
            g_n = g/n_g
            
        alfa = 0
        U = g_n.dot(P)
        c1 = (n**2/(n**2-1)) * (1-alfa**2)
        c2 = (1+n*alfa)/((n+1)*(1+alfa))*2
        x = x - (1+n*alfa)/(n+1)* P.dot(g_n)
        P = c1 * (P - c2 * np.outer(U,U))
        
        
        if np.max(P) <= 1e-6:
            print('obj', f_best, 'sol', x_opt,'grad', x_grad,'P-norm', np.max(P))
            return(x_opt, f_best, x_grad)
    
    print('max iterations reached')

    return x_opt, f_best
                
    
            
    

In [41]:
np.random.seed(3)
c = 4
p = 8
s = 2
l = 4
mu = np.log(1)
sigma = 1
alpha = 0.05
demand = np.random.lognormal(mu,sigma,size= 50000)
frac = alpha* (p-c)/(p-s)
y_nom = y_opt(alpha,p,c,s,l, mu,sigma)


In [35]:
y_nom

1.1422811637881773

In [47]:
a = 1/(2*(sigma*2)**2)
b = 2
alpha = 0.05
x0 = np.array([-5,-12,1,4.2])


In [49]:
radius = np.array([0.001, 0.005, 0.01, 0.02, 0.05, 0.1,0.2,0.4, 0.5])
y_rob = np.zeros(len(radius))
for i in range(len(radius)):
    para = [a,b,alpha,radius[i],c,p,s,l,demand]
    y_rob[i] = ellipsoid_explog_vec(x0, 1000, 100000,para)[0][3]

obj 15.381877670665306 sol [  -4.01051444   -7.98469211 1667.31656559    4.51401351] grad [-3.10994919e-09 -2.42425686e-04 -4.79667688e-11 -4.84856850e-04] P-norm 8.85102677187564e-07
obj 19.336531750249648 sol [ -3.21649579  -9.14285264 688.4711152    5.09253457] grad [ 5.24948307e-10  1.73738273e-04  1.28424274e-09 -1.15542929e-03] P-norm 9.403167996767465e-07
obj 22.118953470096127 sol [ -2.80880627  -9.93547731 464.10541743   5.4881411 ] grad [-1.28508688e-08  1.37224365e-04 -5.05008644e-10 -2.62549445e-03] P-norm 8.510393134046333e-07
obj 25.826424992666897 sol [ -2.28845052 -11.29844039 304.97937555   6.16789182] grad [-2.22637431e-09 -9.42497970e-05  3.22002806e-09 -3.16382172e-04] P-norm 9.408162565670662e-07
obj 32.47602959522528 sol [ -1.59660492 -14.08190082 170.14381751   7.55703006] grad [-3.34106494e-08 -6.76296062e-05 -2.23024134e-08 -1.05902192e-03] P-norm 7.904439058312365e-07
obj 39.1856483335489 sol [ -1.20412827 -16.76439385 109.3802158    8.89368722] grad [ 2.26542

In [51]:
y_rob

array([ 4.51401351,  5.09253457,  5.4881411 ,  6.16789182,  7.55703006,
        8.89368722, 11.50551376, 16.24420414, 19.11486165])